In [1]:
import pandas as pd
import numpy as np
import math
import nltk
from string import punctuation
from nltk.corpus import stopwords

In [2]:
df = pd.read_csv("amazon_alexa.tsv", sep = "\t")

data_dict = {1:{"tokens":[], "text": "", "freq":{}}, 0:{"tokens":[], "text": "", "freq":{}}, "total_text":{"tokens":[], "text": "", "freq":{}}}

for review in df["verified_reviews"]:
    data_dict["total_text"]["text"] += " " + review

for review in df[df["feedback"] == 0]["verified_reviews"]:
    data_dict[0]["text"] += " " + review

for review in df[df["feedback"] == 1]["verified_reviews"]:
    data_dict[1]["text"] += " " + review

In [70]:
stops = stopwords.words('english')
tk = nltk.tokenize.TweetTokenizer()
lemmatizer = nltk.WordNetLemmatizer()
for k in data_dict:
    text = data_dict[k]["text"]
    tokens = tk.tokenize(text)
    lower_tokens = [w.lower() for w in tokens]
    clean_tokens = [lemmatizer.lemmatize(w) for w in lower_tokens if w not in punctuation and w not in stops and len(w)>2]
    freq = nltk.FreqDist(clean_tokens)
    bigrams = list(nltk.ngrams(clean_tokens, 3))
    data_dict[k]["tokens"] = clean_tokens
    data_dict[k]["freq"] = freq
    data_dict[k]["bigrams"] = bigrams
    data_dict[k]["bigrams_freq"] = nltk.FreqDist(bigrams)


In [71]:
word_scores = {1:{}, 0:{}}
bigram_scores = {1:{}, 0:{}}

for k in data_dict:
    if k != "total_text":
        #TODO, tokenizzare le frasi non il testo intero porcodio

        for w in set(data_dict[k]["tokens"]):
            p = data_dict[k]["freq"][w]/len(data_dict[k]["tokens"])
            p_category = data_dict[k]["freq"][w]/data_dict["total_text"]["freq"][w]
            h = -math.log2(p)
            score = h * p_category * p
            word_scores[k][w] = score

        for bigram in set(data_dict[k]["bigrams"]):
            p = data_dict[k]["bigrams_freq"][bigram]/len(data_dict[k]["bigrams"])
            p_category = data_dict[k]["bigrams_freq"][bigram]/data_dict["total_text"]["bigrams_freq"].get(bigram, 1)
            h = -math.log2(p)
            score = p_category * p
            bigram_scores[k][bigram] = score


In [74]:
{k: v for k, v in sorted(bigram_scores[1].items(), key=lambda item: item[1], reverse=True)}

{('love', 'love', 'love'): 0.0007005458419685339,
 ('love', 'echo', 'dot'): 0.0006713564318865116,
 ('love', 'echo', 'show'): 0.0005545987915584227,
 ('easy', 'set', 'use'): 0.0005254093814764004,
 ('still', 'learning', 'use'): 0.00043784115123033366,
 ("i'm", 'still', 'learning'): 0.0004086517411483114,
 ('love', 'fire', 'stick'): 0.0004086517411483114,
 ('love', 'echo', 'spot'): 0.00037946233106628914,
 ('great', 'sound', 'quality'): 0.00037946233106628914,
 ('work', 'great', 'love'): 0.00037946233106628914,
 ('love', 'echo', 'plus'): 0.00035027292098426693,
 ('sound', 'quality', 'great'): 0.00032108351090224467,
 ('alexa', 'turn', 'light'): 0.00032108351090224467,
 ('love', 'work', 'great'): 0.0002955427770804752,
 ('easy', 'set', 'work'): 0.0002918941008202224,
 ('still', 'getting', 'used'): 0.0002918941008202224,
 ('bought', 'prime', 'day'): 0.0002918941008202224,
 ('love', 'echo', 'love'): 0.0002918941008202224,
 ('use', 'echo', 'dot'): 0.00026535827347292946,
 ('great', 'easy', 

In [7]:
df

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1
...,...,...,...,...,...
3145,5,30-Jul-18,Black Dot,"Perfect for kids, adults and everyone in betwe...",1
3146,5,30-Jul-18,Black Dot,"Listening to music, searching locations, check...",1
3147,5,30-Jul-18,Black Dot,"I do love these things, i have them running my...",1
3148,5,30-Jul-18,White Dot,Only complaint I have is that the sound qualit...,1
